In [1]:
import os,sys,time
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from shutil import copy as cp
from glob import glob
import RealSim_IFS

## Information on SDSS-IV MaNGA data

Data Access: https://www.sdss.org/dr16/manga/manga-data/data-access/ 

Data Model: https://www.sdss.org/dr14/manga/manga-data/data-model/

In this test, I download a MaNGA `RSS` file (row-reduced spectra) and generate a cube using `RealSim-IFS`. I then compare the output with the MaNGA DRP `CUBE` for the same object. The output should be similar apart from factors such as astrometric corrections, differential atmospheric refraction (DAR), which are accounted for to ~0.1 arcsec accuracy by the MaNGA DRP.

In [2]:
if 'SLURM_TMPDIR' in [key for key in os.environ.keys()]:
    wdir = os.environ['SLURM_TMPDIR']
    os.chdir(wdir)
    print(os.getcwd())

/localscratch/bottrell.35584111.0


In [29]:

sas_base_dir = 'rsync://data.sdss.org/dr16/manga/spectro/redux/v2_4_3'

plate = 7443
ifudesign = 12703

filename = 'manga-{PLATE}-{IFUDESIGN}-LINRSS.fits.gz'.format(PLATE=plate,IFUDESIGN=ifudesign)
target_url = '{}/{}/stack/{}'.format(sas_base_dir,plate,filename)


if not os.access(filename,0):
    rsync_cmd = 'rsync -avz --info=progress2 {} ./{}'.format(target_url,filename)
    start = time.time()
    os.system(rsync_cmd)
    print(time.time()-start)
   

1432.5272483825684


23.866666666666667
